# **Занятие 6. Свёрточные нейронные сети. Классификация цветных изображений**

https://vk.com/lambda_brain

Рассмотрим случай, когда входные данные -- это цветные изображения. Для обработки таких данных были придуманы **свёрточные нейронные сети**, воспользуемся для этого одной из классических моделей.

---

Импортируем нужные пакеты:

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms


# инициализируем девайс
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Будем обрабатывать стандартный датасет **CIFAR10**, который включает фотографии десяти классов: самолёт, автомобиль, птица, кот, олень, собака, лягушка, лошадь, корабль и грузовик. На каждый класс приходится по 6000 (5000 обучающих и 1000 тестовых) цветных изображений размером 32 * 32 пиксела (и три канала цветности RGB).

In [2]:
input_size = 3*32*32   # Размер изображения в точках * количество цветов
num_classes = 10       # Количество распознающихся классов (10 видов изображений)
n_epochs = 2           # Количество эпох
batch_size = 4         # Размер мини-пакета входных данных
lr = 0.001             # Скорость обучения

Из-за специфики представления цветных изображений мы не можем сразу брать их в сыром виде: сперва требуется нормализовать картинки, превратить их в изображения с интенсивностью цвета в диапазоне 0..1. Это делает стандартная функция **Normalize()** из torchvision.transforms, которой в качестве параметров в подобных случаях обычно указываются стандартные значения (среднее и стандартное отклонение) для такой нормализации (0.5, 0.5, 0.5).

То есть нам нужно выполнить композицию трансформаций: сперва выполнить преобразование в тензоры, и затем нормализовать.

Композицию выполняет стандартная функция torchvision.transforms **.Compose()**. Её результат мы и задаём в качестве параметра transform конструктора CIFAR10.

In [3]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_trainset = dsets.CIFAR10(root='./data', train=True, download=True, transform=transform)


100%|██████████| 170498071/170498071 [00:05<00:00, 30963842.17it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


Аналогично подготовим и тестовый датасет:

In [4]:
cifar_testset = dsets.CIFAR10(root='./data', train=False, download=True, transform=transform)
print(len(cifar_trainset))
print(len(cifar_testset))

Files already downloaded and verified
50000
10000


Загрузим наши данные:

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=cifar_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=cifar_testset,
                                          batch_size=batch_size,
                                          shuffle=False)

Добавим стандартный шаг обучения:

In [6]:
# импортируем нужные библиотеки
import torch
import numpy as np # всегда пригодится :)
from torch.nn import Linear, Sigmoid

# добавляем типовую функцию "шаг обучения"
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

Какую задействовать модель? Вообще, конструирование наиболее эффективных моделей -- это искусство плюс математика. На практике часто можно пользоваться либо простыми стандартными решениями (как было например в случае распознавания рукописных цифр), либо сложными моделями, которые под соответствующие классы задач придумали ведущие специалисты в ML.
В нашем случае мы применим модель LeNet, предложенную Яном ЛеКуном -- она относится к так называемым **свёрточным нейронным сетям (Convolutional Neural Network, CNN)**, хорошо работающим с двумерными изображениями.


---

Подробное описание принципов работы CNN:

https://neurohive.io/ru/tutorial/cnn-na-pytorch/

Главное, обратите внимание на принцип **свёртки** и движущееся окно/**фильтр**.

**Пулинг (pooling)** -- это схожая со скользящим окном техника, когда вместо свёртки по обучаемым весам к значениям в окне применяется некоторая статистическая функция (среднее, максимум, ...). Так, популярная механика тут -- это max pooling. Пулинг выполняет обобщение мелких деталей, устойчиво выделяет некоторый признак независимо от его размера и ориентации.

**Канал** -- это множество фильтров, формирующих оригинальный двумерный вывод. Каналы нередко связываются с цветностью изображения.


In [7]:
import torch.nn as nn
import torch.nn.functional as F


class CifarModel(nn.Module):
    def __init__(self):
        super(CifarModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



Рассмотрим структуру этой модели подробнее.

По мелким техническим причинам (необходимость выравнивания данных при переходе от свёрточных слоёв к линейным) её удобнее представить в формате классической модели, а не просто композицией слоёв.

Первый слой -- Conv2d(3, 6, 5) с функцией активации ReLU создаёт набор свёрточных фильтров. Первый параметр 3 -- это количество входных каналов изображений, три цвета. Второй параметр 6 -- это количество выходных каналов, третий параметр -- размер фильтра 5x5. На выходе получается 6 фильтров размером 3x5x5 -- и всего модель выделяет (3 * 5 * 5 + 1) * 6 = 456 параметров.
Выходной размер слоя получится 6 * 28 * 28 , где 28 = ((32 - 5) + 1)

Метод MaxPool2d(2,2) -- это реализация max-пулинга (вычисление его аргументов см. по ссылке выше). kernel_size -- размер окна пулинга, stride -- шаг пулинга. Выходной размер слоя таким образом снижаем в два раза: с 6 * 28 * 28 до 6 * 14 * 14.


Далее снова применяется функция Conv2d(6, 16, 5) -- шесть выходных каналов предыдущей функций как входы. Теперь мы применяем 16 фильтров (каждый размером 6 * 5 * 5), и выходной размер слоя будет 16 * 10 * 10, где 10 = (14 - 5) + 1. Всего на уровне обрабатывается (5 * 5 * 6 + 1) * 16 = 2416 параметров.

Следующий max pooling снижает этот выход в два раза -- с 16 * 10 * 10 до 16 * 5 * 5.

И в заключение добавляются три полносвязных слоя Linear. Обратите внимание, что перед ними надо выполнить модификацию структуры передаваемых данных, так как свёрточные слои работают с двумерными изображениями, а линейные -- с векторными наборами. Такое преобразование выполняет x.view(-1, 16 * 5 * 5).

Первый линейный слой из 120 узлов, получает 16 * 5 * 5 входов -- то есть в нём требуется (16 * 5 * 5 + 1) * 120 = 48120 параметров, и далее количество входов-выходов понижается через следующие слои до наших итоговых 10 классов (последний уровень требует (84+1) * 10 = 850 параметров).


In [8]:
from torch import optim, nn

model = CifarModel()
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        loss = train_step(images, labels)
    print(f"{epoch=} {loss=}")

# print(model.state_dict())
# print(loss)

epoch=0 loss=1.0366183519363403
epoch=1 loss=1.3498492240905762


In [9]:
with torch.no_grad(): # проверяем на тестовой выборке
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Точность: {} %'.format(100 * correct / total))

Точность: 53.51 %


Считаем точность -- она получается в районе 55%. Это хороший результат, так как при случайном выборе мы получили бы 10%. Причём её можно существенно повысить -- уже после 10 эпох обучения мы получим точность 80+%!


---



В заключение вычислим точность распознавания по каждому из признаков:

In [10]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Точность для %5s : %2d %%' % (
        labels[i], 100 * class_correct[i] / class_total[i]))

Точность для tensor(3, device='cuda:0') : 62 %
Точность для tensor(5, device='cuda:0') : 82 %
Точность для tensor(1, device='cuda:0') : 34 %
Точность для tensor(7, device='cuda:0') : 24 %


## **Задание.**
Увеличение количества эпох (шагов обучения) существенно повышает качество модели. Но есть и другие способы -- поэкспериментируйте например с добавлением новых свёрточных или линейных слоёв, размерами фильтров и их количеством. В процессе экспериментов избегайте переобучения -- когда модель показывает отличные результаты на обучающей выборке, но невысокие на тестовой.


---

В следующем занятии мы займёмся экспериментами с уже обученными моделями.

In [11]:
input_size = 3*32*32   # Размер изображения в точках * количество цветов
num_classes = 10       # Количество распознающихся классов (10 видов изображений)
n_epochs = 10          # Количество эпох
batch_size = 4         # Размер мини-пакета входных данных
lr = 0.001             # Скорость обучения

In [12]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_trainset = dsets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = dsets.CIFAR10(root='./data', train=False, download=True, transform=transform)
print(len(cifar_trainset))
print(len(cifar_testset))

Files already downloaded and verified
Files already downloaded and verified
50000
10000


In [13]:
train_loader = torch.utils.data.DataLoader(dataset=cifar_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=cifar_testset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [14]:
from torch import optim, nn

model = CifarModel()
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        loss = train_step(images, labels)
    print(f"{epoch=} {loss=}")

epoch=0 loss=1.4364426136016846
epoch=1 loss=1.9631304740905762
epoch=2 loss=1.7222373485565186
epoch=3 loss=0.9289392828941345
epoch=4 loss=2.3496437072753906
epoch=5 loss=0.5360375046730042
epoch=6 loss=1.0796667337417603
epoch=7 loss=1.7447887659072876
epoch=8 loss=1.0040431022644043
epoch=9 loss=0.25247350335121155


In [15]:
with torch.no_grad(): # проверяем на тестовой выборке
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Точность: {} %'.format(100 * correct / total))

Точность: 60.58 %


In [16]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Точность для %5s : %2d %%' % (
        labels[i], 100 * class_correct[i] / class_total[i]))

Точность для tensor(3, device='cuda:0') : 65 %
Точность для tensor(5, device='cuda:0') : 70 %
Точность для tensor(1, device='cuda:0') : 48 %
Точность для tensor(7, device='cuda:0') : 34 %


**ResNet**

Поскольку не удалось добиться точности свыше 62%, попробуем использовать ResNet

Имплементация на PyTorch + четкое объяснение принципов работы ResNet
https://debuggercafe.com/implementing-resnet18-in-pytorch-from-scratch/

Я слегка переписал имплементацию на более правильную версию с точки зрения ООП (как я вижу, могу ошибаться)

https://github.com/Xavatu/ML/blob/master/resnet.py

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms


# инициализируем девайс
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
import torch
import torch.nn as nn


class Block(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int,
        expansion: int,
        downsample: nn.Module = None,
    ) -> None:
        super(Block, self).__init__()
        raise NotImplementedError

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        raise NotImplementedError


class BasicBlock(Block):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int,
        expansion: int,
        downsample: nn.Module = None,
    ) -> None:
        super(Block, self).__init__()
        self.downsample = downsample
        self.expansion = expansion
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels * self.expansion,
            kernel_size=3,
            padding=1,
            bias=False,
        )
        self.batch_norm2 = nn.BatchNorm2d(out_channels * self.expansion)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.downsample(x) if self.downsample is not None else x
        out = self.conv1(x)
        out = self.batch_norm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batch_norm2(out)
        out += identity
        out = self.relu(out)
        return out


class BottleNeckBlock(Block):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int,
        expansion: int,
        downsample: nn.Module = None,
    ) -> None:
        super(Block, self).__init__()
        self.downsample = downsample
        self.expansion = expansion
        self.relu = nn.ReLU()
        self.conv0 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=1,
            stride=1,
            bias=False,
        )
        self.batch_norm0 = nn.BatchNorm2d(out_channels)
        in_channels = out_channels
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels * self.expansion,
            kernel_size=1,
            stride=1,
            bias=False,
        )
        self.batch_norm2 = nn.BatchNorm2d(out_channels * self.expansion)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.downsample(x) if self.downsample is not None else x
        out = self.conv0(x)
        out = self.batch_norm0(out)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.batch_norm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batch_norm2(out)
        out += identity
        out = self.relu(out)
        return out


class _ResNet(nn.Module):
    def __init__(
        self,
        img_channels: int,
        block: type[Block],
        num_classes: int,
        blocks_count: tuple[int, int, int, int],
        expansion: int,
    ) -> None:
        super(_ResNet, self).__init__()
        self.blocks_count = blocks_count
        self.expansion = expansion
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            in_channels=img_channels,
            out_channels=self.in_channels,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False,
        )
        self.batch_norm = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(
            block=block, out_channels=64, count=self.blocks_count[0], stride=1
        )
        self.layer2 = self._make_layer(
            block=block, out_channels=128, count=self.blocks_count[1], stride=2
        )
        self.layer3 = self._make_layer(
            block=block, out_channels=256, count=self.blocks_count[2], stride=2
        )
        self.layer4 = self._make_layer(
            block=block, out_channels=512, count=self.blocks_count[3], stride=2
        )
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * self.expansion, num_classes)

    def _make_layer(
        self,
        block: type[Block],
        out_channels: int,
        count: int,
        stride: int,
    ) -> nn.Sequential:
        downsample = None
        if stride != 1 or self.in_channels != out_channels * self.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(
                    in_channels=self.in_channels,
                    out_channels=out_channels * self.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels * self.expansion),
            )
        layers = [
            (
                block(
                    self.in_channels,
                    out_channels,
                    stride,
                    self.expansion,
                    downsample,
                )
            )
        ]
        self.in_channels = out_channels * self.expansion
        layers += [
            block(
                in_channels=self.in_channels,
                out_channels=out_channels,
                stride=1,
                expansion=self.expansion,
            )
            for _ in range(count - 1)
        ]
        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.adaptive_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


class ResNet18(_ResNet):
    def __init__(
        self,
        img_channels: int,
        num_classes: int,
    ):
        super().__init__(
            img_channels=img_channels,
            block=BasicBlock,
            num_classes=num_classes,
            blocks_count=(2, 2, 2, 2),
            expansion=1,
        )


class ResNet34(_ResNet):
    def __init__(
        self,
        img_channels: int,
        num_classes: int,
    ):
        super().__init__(
            img_channels=img_channels,
            block=BasicBlock,
            num_classes=num_classes,
            blocks_count=(3, 4, 6, 3),
            expansion=1,
        )


class ResNet50(_ResNet):
    def __init__(
        self,
        img_channels: int,
        num_classes: int,
    ):
        super().__init__(
            img_channels=img_channels,
            block=BottleNeckBlock,
            num_classes=num_classes,
            blocks_count=(3, 4, 6, 3),
            expansion=4,
        )


class ResNet101(_ResNet):
    def __init__(
        self,
        img_channels: int,
        num_classes: int,
    ):
        super().__init__(
            img_channels=img_channels,
            block=BottleNeckBlock,
            num_classes=num_classes,
            blocks_count=(3, 4, 23, 3),
            expansion=4,
        )


class ResNet152(_ResNet):
    def __init__(
        self,
        img_channels: int,
        num_classes: int,
    ):
        super().__init__(
            img_channels=img_channels,
            block=BottleNeckBlock,
            num_classes=num_classes,
            blocks_count=(3, 8, 36, 3),
            expansion=4,
        )

In [55]:
# добавляем типовую функцию "шаг обучения"
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return yhat, loss.item()
    return train_step


In [67]:
input_size = 3*32*32   # Размер изображения в точках * количество цветов
num_classes = 10       # Количество распознающихся классов (10 видов изображений)
n_epochs = 100         # Количество эпох
batch_size = 256       # Размер мини-пакета входных данных
lr = 0.01              # Скорость обучения

In [68]:
import torchvision.transforms as transforms

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
cifar_trainset = dsets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
cifar_testset = dsets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
print(len(cifar_trainset))
print(len(cifar_testset))

Files already downloaded and verified
Files already downloaded and verified
50000
10000


In [69]:
train_loader = torch.utils.data.DataLoader(dataset=cifar_trainset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=cifar_testset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2)

In [70]:
from torch import optim, nn

model = ResNet18(
    img_channels=3,
    num_classes=10,
)
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    train_running_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs, loss = train_step(images, labels)
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
    accuracy = 100. * (train_running_correct / len(train_loader.dataset))
    print(f"{epoch=} {loss=} {accuracy=}")

epoch=0 loss=1.5492775440216064 accuracy=40.364
epoch=1 loss=1.555121660232544 accuracy=51.856
epoch=2 loss=0.9267351031303406 accuracy=56.69
epoch=3 loss=1.047095775604248 accuracy=60.284000000000006
epoch=4 loss=1.1124536991119385 accuracy=62.864
epoch=5 loss=1.0063002109527588 accuracy=65.034
epoch=6 loss=0.8574200868606567 accuracy=66.682
epoch=7 loss=1.0344228744506836 accuracy=68.662
epoch=8 loss=0.8414037823677063 accuracy=69.834
epoch=9 loss=0.5436530113220215 accuracy=71.36200000000001
epoch=10 loss=0.8747626543045044 accuracy=72.55
epoch=11 loss=0.8477487564086914 accuracy=73.844
epoch=12 loss=0.7753360271453857 accuracy=75.102
epoch=13 loss=0.4889650344848633 accuracy=75.952
epoch=14 loss=0.6370298862457275 accuracy=76.584
epoch=15 loss=0.822266697883606 accuracy=77.91799999999999
epoch=16 loss=0.39303797483444214 accuracy=78.634
epoch=17 loss=0.6396672129631042 accuracy=79.644
epoch=18 loss=0.43366527557373047 accuracy=80.458
epoch=19 loss=0.6026569604873657 accuracy=81.033

KeyboardInterrupt: 

In [71]:
with torch.no_grad(): # проверяем на тестовой выборке
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Точность: {} %'.format(100 * correct / total))

Точность: 75.71 %
